# **Elements of Artificial Intelligence and Data Science 24/25**
## **Exercise Sheet 6: Supervised Learning and Performance Evaluation**

In this notebook, you will learn to handle some supervised learning tasks using the `hcc.csv` dataset available on Moodle.
Here, some template code has already been provided for you, and it will be your task to implement the additional functionality.

- Sections that begin with **[Implementation]** indicate that the following block of code will require additional functionality which you must provide.

- There might also be some suggested **[Questions]** to help you analyse and discuss the obtained results.

## **Exploring the Data**
Start by loading the file and understanding characteristics of the data and the features.

##### **[Questions]**
- How many patterns/observations are there? How many features?
- What are the types and possible values of the features? 
- What is the class of this problem?

In [147]:
# At the end of the project, you can add here all necessary imports
import pandas as pd

In [148]:
# Read hcc data
df = pd.read_csv("hcc.csv")
df

,Gender,Alcohol,Age,Encefalopathy,Total_Bil,Location,CLASS
0,M,Y,67,No,2.1,Seg I,LIVES
1,F,N,62,No,NaN,Seg I,LIVES
2,M,Y,78,No,0.4,Seg V,LIVES
3,M,Y,77,No,0.4,Seg II,DIES
4,M,Y,76,No,0.7,Seg I,LIVES
...,...,...,...,...,...,...,...
160,F,Y,40,No,0.5,Seg V,LIVES
161,F,N,68,No,3.6,Seg I,DIES
162,M,Y,65,No,0.6,Seg V,LIVES
163,M,Y,44,No,1.1,Seg V,DIES


##### **[Implementation] Data Exploration**
Let's begin by investigating the dataset to determine how many patients we have information on, and learn about the mortality rate among these patients. In the code cell below, you will need to compute the following:
- The total number of patients, `n_patients`.
- The total number of features for each patient, `n_features`.
- The number of those patients who lived, `n_lives`.
- The number of those patients who died, `n_dies`.
- The mortality rate of the class, `mortality_rate`, in percent (%).

In [149]:
# TODO: Calculate number of patients
n_patients = len(df)

# TODO: Calculate number of features
n_features = len(df.columns)-1

# TODO: Calculate patients that survived
n_lives = len(df[df["CLASS"]=="LIVES"])

# TODO: Calculate patients that died
n_dies = len(df[df["CLASS"]=="DIES"])

# TODO: Calculate mortality rate
mortality_rate = (n_dies/n_patients)*100

# Print the results
print(f"Total number of patients: {n_patients}")
print(f"Total number of features: {n_features}")
print(f"Number of patients that survived: {n_lives}")
print(f"Number of patients that died: {n_dies}")
print(f"Mortality rate: {mortality_rate:.2%}")

Total number of patients: 165
Total number of features: 6
Number of patients that survived: 102
Number of patients that died: 63
Mortality rate: 3818.18%


##### **[Questions]**
- How can you further investigate this dataset? Refer back to the **Lab05: Data Cleaning and Preprocessing** and try to use **Feature Assessment** and **Visualization** to find good discriminative features. Use histogramss, boxplots, heatmaps
- You can also explore existing problems in this dataset. Does the dataset have missing values? Outliers? Correlated features? Imbalanced Classes? Other issues?

    Falta de Valores: Algumas linhas de Total_Bil e Location estão vazias.

    Outliers: Valores muito altos ou baixos, como Total_Bil com 40.5 ou 28.9, que são bem diferentes dos outros.

    Classes desbalanceadas: Temos mais pacientes que sobreviveram do que morreram, o que pode dificultar as previsões.

    Features correlacionadas: Talvez idade e encefalopatia tenham relação, e precisamos de ver.

**Hint:** Install and explore tools such as [ydata-profiling](https://docs.profiling.ydata.ai/latest/) and [dataprep](https://docs.dataprep.ai/user_guide/user_guide.html) to explore this dataset in a more flexible way.

##### **[Implementation] Data Quality**
This dataset has a few points with missing values in 2 features. Find out which and impute them. For simplicity, you can use [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer). 

**Note:** Pay attention to the type of features you have to choose the most appropriate strategy (e.g. categorical features don't have mean values...)

In [150]:
# TODO: Import any additional functionality you may need here
from sklearn.impute import SimpleImputer

# TODO: Impute missing values with mean/mode using SimpleImputer

print("Valores faltando por coluna:")
print(df.isna().sum())

imputer_numeric = SimpleImputer(strategy="mean")
df["Total_Bil"] = imputer_numeric.fit_transform(df[["Total_Bil"]])

imputer_categorical = SimpleImputer(strategy="most_frequent")
df["Location"] = imputer_categorical.fit_transform(df[["Location"]]).ravel()

print("\nValores faltando por coluna depois do impute:")
print(df.isna().sum())

#Para Total_Bil, calculamos a média dos valores existentes e colocamos nos lugares vazios.
#Para Location, escolhemos o valor mais comum (ex.: "Seg I") e colocamos nos lugares vazios.

Valores faltando por coluna:
Gender           0
Alcohol          0
Age              0
Encefalopathy    0
Total_Bil        5
Location         2
CLASS            0
dtype: int64

Valores faltando por coluna depois do impute:
Gender           0
Alcohol          0
Age              0
Encefalopathy    0
Total_Bil        0
Location         0
CLASS            0
dtype: int64


## **Preparing the Data**
It is often the case that the data you obtain contains non-numeric features. This can be a problem since some machine learning algorithms expect numeric data to perform computations with. 

### **[Implementation] Preprocess Feature Columns**

As you can see, there are several non-numeric columns that need to be converted:

- Some of them are simply `M`/`F` or `Y`/`N` (e.g. `Gender` or`Alcohol`). These can be reasonably converted into `1`/`0` (binary) values. We can use a simple `map` for this (check the cell below).

- Other columns, like `Encefalopathy` and `Location`, have more than two values:
    - Some might be _ordinal_, where there is an ordering between values. `Encefalopathy` can be one of these cases: `No` < `Grade I/II` < `Grade III/IV`. We can handle them by using a [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) function from scikit-learn. 
    - Others, like `Location` are _nominal_ because there is no ordering between values. The recommended way to handle such columns is to create as many columns as possible values (e.g. `Seg I`, `Seg II`, `Seg III`, etc.), and assign a `1` to one of them and `0` to all others. That is called **OneHotEncoding** and we can use the scikit-learn [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [151]:
# Map the M/F and Y/N to 1/0 values
GENDER_LABELS = {"M": 0, "F": 1}
ALCOHOL_LABELS = {"N": 0, "Y": 1}

df.loc[:, "Gender"] = df["Gender"].map(GENDER_LABELS)
df.loc[:, "Alcohol"] = df["Alcohol"].map(ALCOHOL_LABELS)

In [152]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# TODO: Transform Encephalopathy using Ordinal Encoder
oe = OrdinalEncoder()
df["Encefalopathy"] = oe.fit_transform(df[["Encefalopathy"]])


In [153]:
# Transform Location using One Hot Encoder
ohe = OneHotEncoder(sparse_output=False)
location_ohe = ohe.fit_transform(df[["Location"]])

In [154]:
location_encoded = pd.DataFrame(location_ohe, columns=ohe.get_feature_names_out(["Location"]))

In [155]:
df_encoded = pd.concat([df, location_encoded], axis=1)  # Concatenate both frames

In [156]:
df_encoded.drop(columns=["Location"], inplace=True)     # Drop "Location" feature (we have now the OneHotEncoding version)

# Put "CLASS" (target) again at the end of the dataframe
cols = list(df_encoded.columns)
cols.remove("CLASS")
cols.append("CLASS")
df_encoded = df_encoded[cols]

You may now want to normalize your data to respect the `[0, 1]` range for all features. Consider using [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) from scikit-learn. You should also normalize `Encephalopathy`.

Finally, maybe consider a mapping to encode the `CLASS`. Remember that you want to predict the "mortality" so consider `Dies = 1` as you "positive" class (the one you'd like to predict).

In [157]:
# TODO: Import any additional functionality you may need here
from sklearn.preprocessing import MinMaxScaler
# TODO: Encode the class labels: Lives and Dies into 0 and 1, respectively
df_encoded["CLASS"] = df_encoded["CLASS"].map({"LIVES": 0, "DIES": 1})
# TODO: Normalize the Age, Total_Bil, and Encephalopathy columns using MinMaxScaler
scaler = MinMaxScaler()
columns_to_scale = ["Age", "Total_Bil", "Encefalopathy"]
df_encoded[columns_to_scale] = scaler.fit_transform(df_encoded[columns_to_scale])

In [158]:
df_encoded.describe()

,Age,Encefalopathy,Total_Bil,Location_Seg I,Location_Seg II,Location_Seg III,Location_Seg IV,Location_Seg V,CLASS
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,0.612204,0.878788,0.069352,0.418182,0.145455,0.066667,0.012121,0.357576,0.381818
std,0.182459,0.317918,0.134699,0.494762,0.353632,0.250203,0.109760,0.480745,0.487311
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.506849,1.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.630137,1.000000,0.027363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.739726,1.000000,0.069352,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## **Training and Testing Data Split**
By now, we should have our dataset preprocessed and ready for the classification task. For the next step, we will split the data (both features and corresponding labels) into training and test sets using the **Hold-out** (check the course slides to remember what this is).

##### **[Implementation] Holdout Method for Train/Test split**
To implement the hold-out method, we will use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from scikit-learn, keeping 80% for training and 20% for the test set.

**Note:** This is a very naive approach to data partitioning. In your project, you should either considering using hold-out several times (and average the results), or consider more advanced strategies such as k-fold cross-validation. If you try to perform hold-out several times and average the results, don't forget to change/remove the `random_state`!)

In [159]:
# Divide your dataframe into X (features) and class (y)
X = df_encoded.iloc[:,:-1].astype(float)
y = df_encoded.iloc[:, -1].astype(int)

In [160]:
X

,Gender,Alcohol,Age,Encefalopathy,Total_Bil,Location_Seg I,Location_Seg II,Location_Seg III,Location_Seg IV,Location_Seg V
0,0.0,1.0,0.643836,1.0,0.044776,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.575342,1.0,0.069352,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.794521,1.0,0.002488,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.780822,1.0,0.002488,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.767123,1.0,0.009950,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
160,1.0,1.0,0.273973,1.0,0.004975,0.0,0.0,0.0,0.0,1.0
161,1.0,0.0,0.657534,1.0,0.082090,1.0,0.0,0.0,0.0,0.0
162,0.0,1.0,0.616438,1.0,0.007463,0.0,0.0,0.0,0.0,1.0
163,0.0,1.0,0.328767,1.0,0.019900,0.0,0.0,0.0,0.0,1.0


In [161]:
y

0      0
1      0
2      0
3      1
4      0
      ..
160    0
161    1
162    0
163    1
164    1
Name: CLASS, Length: 165, dtype: int64

In [162]:
X.dtypes

Gender              float64
Alcohol             float64
Age                 float64
Encefalopathy       float64
Total_Bil           float64
Location_Seg I      float64
Location_Seg II     float64
Location_Seg III    float64
Location_Seg IV     float64
Location_Seg V      float64
dtype: object

In [163]:
y.dtype

dtype('int64')

In [164]:
from sklearn.model_selection import train_test_split

# TODO: Set the X_train, X_test, y_train, y_test variables according to 80/20 split
X = df_encoded.iloc[:, :-1].astype(float)
y = df_encoded.iloc[:, -1].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Show the results of the split
print(f"Training set has {X_train.shape[0]} patterns: {y_train.value_counts()[0]} from class 0 and {y_train.value_counts()[1]} from class 1.")
print(f"Training set has {X_test.shape[0]} patterns: {y_test.value_counts()[0]} from class 0 and {y_test.value_counts()[1]} from class 1.")
print(f"The sum of patterns is {X_train.shape[0] + X_test.shape[0]} patterns, which amount to the original {X.shape[0]} patterns.")

Training set has 132 patterns: 82 from class 0 and 50 from class 1.
Training set has 33 patterns: 20 from class 0 and 13 from class 1.
The sum of patterns is 165 patterns, which amount to the original 165 patterns.


## **Training and Evaluating Classification Models**
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You should first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. Then, you will then fit the model to the training data and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the test set.

**The [following supervised learning models](http://scikit-learn.org/stable/supervised_learning.html) are currently available in `scikit-learn` that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression


##### **[Questions]**
List three supervised learning models that are appropriate for this problem. For each model chosen:

- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

##### **[Implementation] Classifying with K-Nearest Neighbours**
For the purpose of this exercise, we will use [KNN - Nearest Neighbours Classification](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification). 

You can explore the code cell below that creates three helper functions which you can use for training and testing supervised learning. The functions are as follows:
- `train_classifier` - takes as input a classifier (`clf`) and training data (`X_train`) and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier (`clf`), the test data (`X_test`) and its grouth truth (`y_test`), makes predictions about the `X_test` and computes the F<sub>1</sub> score by comparing the predicted labels (`y_test`) with the grouth truth (`y_pred`).
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [165]:
from time import time
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    """Fits a classifier to the training data."""
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print(f"Trained model in {(end-start):.4f} seconds")

def predict_labels(clf, X_test, y_test):
    """Makes predictions using a fit classifier and calculates the F1 score."""
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(X_test)
    end = time()
    
    # Print and return results
    print(f"Made predictions in {(end-start):.4f} seconds.")
    return f1_score(y_test, y_pred, pos_label=1) # Positive label is 1 ("Dies")

def train_predict(clf, X_train, y_train, X_test, y_test):
    """Training and evaluating a classifier usig F1 score."""

    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and test sets
    print(f"F1 score for training set: {predict_labels(clf, X_train, y_train):.4f}.")
    print(f"F1 score for test set: {predict_labels(clf, X_test, y_test):.4f}.")

In [169]:
# TODO: Import the KNeighborsClassifier and use train_predict to see the results
from sklearn.neighbors import KNeighborsClassifier

# Criar o modelo KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)

# Treinar e prever
train_predict(knn_clf, X_train, y_train, X_test, y_test)

Trained model in 0.0055 seconds
Made predictions in 0.0060 seconds.
F1 score for training set: 0.5000.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.4000.


## **Evaluating Performance Metrics**

With the predefined functions above, you can import the three supervised learning models of your choice and run the `train_predict` function for each one.

**Note that:**

- Depending on the chosen classifiers, you might need to perform a slightly different preprocessing (*most classifiers in scikit-learn can handle numeric data as it is, but to improve interpretability, it might be more adequate to transform the features in different ways*).

- You can initialize the three learning models and store them as `clf_A`, `clf_B`, `clf_C`, etc. Or give them more specific names, such as `knn_clf`, `df_clf`, etc. `clf` stands for "classifier", but it is just a convention and you can give it any other name!

- You can use a `random_state` for each model you create, to ensure its initialization is the same between runs (*this ensures that model predictions are reproducible*).

- For now, you can use the default settings for each model — you will tune one specific model in a later section.
 
- You can use the table below to store your results.

In [172]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = KNeighborsClassifier(n_neighbors=5)
clf_B = DecisionTreeClassifier(random_state=42)
clf_C = LogisticRegression(random_state=42)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
print("KNN Classifier:")
train_predict(clf_A, X_train, y_train, X_test, y_test)
print("\nDecision Tree Classifier:")
train_predict(clf_B, X_train, y_train, X_test, y_test)
print("\nLogistic Regression Classifier:")
train_predict(clf_C, X_train, y_train, X_test, y_test)

KNN Classifier:
Trained model in 0.0020 seconds
Made predictions in 0.0067 seconds.
F1 score for training set: 0.5000.
Made predictions in 0.0026 seconds.
F1 score for test set: 0.4000.

Decision Tree Classifier:
Trained model in 0.0048 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.4444.

Logistic Regression Classifier:
Trained model in 0.0103 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.2462.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.2353.


### **Tabular Results**
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the table provided.

| Classifier        | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| NAME Classifier 1 |                         |                        |                  |                 |
| NAME Classifier 2 |        EXAMPLE          |                        |                  |                 |
| NAME Classifier 3 |                         |                        |                  |    EXAMPLE      |

## **Choosing the Best Model**
In this final section, you will choose from the three supervised learning models the *best* model to use on the HCC data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

##### **[Questions]**

- Based on your experiments how would you explain to the board of supervisors which model you chose as the best model? Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

- In your own words, how would you explain to the board of directors how the final model chosen is supposed to work? Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Imagina that the directors known nothing about machine learning, so avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.

## **Hyperparameter Tuning**
Once you've chosen a model, you can try to optimize your results by *looking for a set of parameters* that increase your performance metric. To do this, you can use [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In the following example, we will use `GridSearchCV` to tune the performance of a `KNeighborsClassifier`.




In [181]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

print(f"Current parameters and values for our previous KNN classifier:\n{clf_A.get_params()}")

params = [{'n_neighbors': [3, 5, 7, 9],
           'weights': ['uniform', 'distance'],
           'metric': ['euclidean', 'cosine', 'cityblock', 'manhattan']
           }]

knn = KNeighborsClassifier()
gs_knn = GridSearchCV(knn, param_grid=params, scoring="f1")
gs_knn.fit(X_train, y_train)

print(f"\nGrid Search KNN Best Parameters:\n {gs_knn.best_params_}")
print(f"\nBest Model F1 Score: {gs_knn.best_score_}")

# Use the best model directly
print("\nResults from best KNN Model:")
best_knn_clf = gs_knn.best_estimator_
train_predict(best_knn_clf, X_train, y_train, X_test, y_test)

Current parameters and values for our previous KNN classifier:
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}

Grid Search KNN Best Parameters:
 {'metric': 'cosine', 'n_neighbors': 3, 'weights': 'distance'}

Best Model F1 Score: 0.39953216374269

Results from best KNN Model:
Trained model in 0.0006 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.5833.


##### **[Implementation] Experimenting with Hyperparameter Tuning**

You can now use grid search (`GridSearchCV`) to tune some parameters of your previously chosen models. 

You can start by filling the code cell below to specify the grid search configurations for another of your classifiers:

- Import the necessary functionality (classifier, gridsearch, and metrics)
- Create a dictionary of parameters you wish to tune for the chosen model
- Initialize the classifier `clf`
- Perform grid search on the classifier `clf` using `f1` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`)
- Find the `best_estimator` (best classifier) that contains the best parameters
- Test the best classifier on the training and test sets

In [182]:
# TODO: Import all necessary functionality here
from sklearn.model_selection import GridSearchCV
# TODO: Create the parameters list you wish to tune
parameters = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "cosine", "manhattan"]
}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Perform grid search on the classifier using the "f1" as the scoring method
grid_obj = None

# TODO: Fit the grid search object on the training data and find the optimal parameters

grid_obj = GridSearchCV(clf, param_grid=parameters, scoring="f1")
grid_obj.fit(X_train, y_train)

# TODO: Get the best estimator

best_clf = grid_obj.best_estimator_

# TODO: Report the final F1 score for training and testing after parameter tuning

print(f"Best Parameters: {grid_obj.best_params_}")
print(f"Best F1 Score: {grid_obj.best_score_}")
train_predict(best_clf, X_train, y_train, X_test, y_test)

Best Parameters: {'metric': 'cosine', 'n_neighbors': 3, 'weights': 'distance'}
Best F1 Score: 0.39953216374269
Trained model in 0.0006 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.5833.


##### **[Questions]**

- What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?


##### **[Further Work]**

This notebook considers a gentle introduction to a standard end-to-end machine learning pipeline. In real-world domains, you might face more complex challenges (*more features and patterns, more complex data issues, decentralized data sources, and so on*). There are many other experiments you might want to do with this data, such as:

* Considering the original [Hepatocellular Carcinoma Dataset](https://www.kaggle.com/datasets/mrsantos/hcc-dataset). How would your approach need to change?
* Considering using a custom split of the data, using a different number of training points each time (e.g., 25% versus 75% of points for training). How would the performance results vary?
* Experiment with distinct missing data imputation strategies and/or imbalanced data strategies? Which one(s) yield(s) the best results?
* You've also been doing a lot of scattered scripts/code cells! Can you refactor your code to automate the process in a clear and clean way?
* What kind of other experiments can you think of? How can data science/machine learning techniques further support the exploration of this dataset?


## **Bibliography**

- [Using pandas and Python to Explore Your Dataset](https://realpython.com/pandas-python-explore-dataset/) by Reka Horvath in The Real Python (*try to open it in a Private Browser, if a blocking window appears*).
- [scikit-learn.org: Machine Learning in Python](https://scikit-learn.org/stable/)
- [10 Free Machine Learning Books for 2025](https://www.kdnuggets.com/10-free-machine-learning-books-for-2025), KDNuggets
- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/): Online Book
- [Machine Learning Pocket Reference](https://github.com/mattharrison/ml_pocket_reference): Source code to complement the book available on Moodle